In [1]:
from hmmlearn import hmm
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.preprocessing import MinMaxScaler
from matplotlib import cm, pyplot as plt
import pickle
import warnings
from math import sqrt
import numpy
import pyedflib
import numpy as np
import time
start_time = time.time()
np.random.seed(111)


In [2]:

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from pandas import datetime
import os
import pandas as pd
import time
import warnings
import numpy as np
from numpy import newaxis
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import pyedflib
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy Numpy warnings


# scale train and test data to [-1, 1]
def scale_t(data):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    data = data.reshape(-1, 1)
    #scaler = StandardScaler()
    scaler = scaler.fit(data)
    # transform data
    print(data.shape[0])
    
    data_scaled = scaler.transform(data)
    
    return scaler, data_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X):
    X = X.reshape(-1,1)
    inverted = scaler.inverse_transform(X)
    return inverted[:, 0]

def load_data(filename):
    g = pyedflib.EdfReader(filename)
    n = g.signals_in_file
    signal_labels = g.getSignalLabels()
    sig = np.zeros((n, g.getNSamples()[0]))
    for i in np.arange(n):
        len_sig = len(g.readSignal(i))
        sig[i, :len_sig] = g.readSignal(i)
    return sig


def chunk_data(diff, sequence_length):
    chunk_data = np.zeros((diff.shape[0],diff.shape[1] - sequence_length, sequence_length))
    for index in range(diff.shape[1] - sequence_length):
        for k in range(diff.shape[0]):
            chunk_data[k,index] = diff[k,index: index + sequence_length]
    print(chunk_data[0,0])
    return chunk_data

def difference(datset, interval):
    diff = np.zeros((dataset.shape[0],dataset.shape[1]))
    for k in range(dataset.shape[0]):
        for i in range(interval, dataset.shape[1]):
            diff[k,i] = dataset[k,i] - dataset[k,i - interval]
    return diff
            

# create a differenced series
def normalize_all(result): 
    dataset = np.zeros(result.shape)
    for i in range(result.shape[0]):
        scaler, data_s = scale_t(result[i])
        dataset[i] = data_s[:,0]
    return dataset


def make_data(sig):
    data = sig[:3,:360000*2]
    data_scale = np.zeros(data.shape)
    scalers = []
    for i in range(data.shape[0]):
        scaler, data_s = scale_t(data[i])
        data_scale[i] = data_s[:,0]
        scalers.append(scaler)
    
    row = round(0.9 * data_scale.shape[1])
    data_scale = data_scale.T
    train = data_scale[:int(row), :]
    test = data_scale[int(row):, :]
    
    
    #print(train_scaled.shape,test_scaled.shape)  
    #print(train_scaled[0],test_scaled[0])
    #np.random.shuffle(train)
    return [scalers, train, test]

/opt/conda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import matplotlib.pyplot as plt

def plot_results(predicted_data, true_data, make_line):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    if make_line:
        plt.axvline(x=y_train.shape[0] + 1, linestyle='--', color = 'b', label = 'future predicted points')
    plt.show()

In [4]:
print('> Loading data... ')
sig = load_data('LSTM/sample-data/SC4002E0-PSG.edf')
print("--- %s seconds ---" % (time.time() - start_time))

> Loading data... 
--- 4.5526638031 seconds ---


In [5]:
scalers, train, test = make_data(sig)
print(train.shape, test.shape)

720000
720000
720000
((648000, 3), (72000, 3))


In [6]:
model_3 = hmm.GaussianHMM(n_components=4, covariance_type="diag", n_iter = 500, verbose=True).fit(train)
print(model_3.means_)
print(model_3.covars_)
print(model_3.startprob_)
print(model_3.transmat_)
z = model_3.predict(train)
print z.shape

        47     1671926.2345          +0.3196
        48     1671926.5395          +0.3050
        49     1671926.8310          +0.2915
        50     1671927.1099          +0.2789
        51     1671927.3770          +0.2671
        52     1671927.6330          +0.2560
        53     1671927.8785          +0.2456
        54     1671928.1142          +0.2357
        55     1671928.3406          +0.2264
        56     1671928.5582          +0.2175
        57     1671928.7673          +0.2092
        58     1671928.9686          +0.2012
        59     1671929.1623          +0.1937
        60     1671929.3487          +0.1865
        61     1671929.5283          +0.1796
        62     1671929.7014          +0.1731
        63     1671929.8681          +0.1668
        64     1671930.0289          +0.1608
        65     1671930.1840          +0.1551
        66     1671930.3337          +0.1496
        67     1671930.4781          +0.1444
        68     1671930.6175          +0.1394
        69

[[ 0.00383045 -0.05216758 -0.01588151]
 [ 0.0231715  -0.03092811 -0.24610446]
 [ 0.01513188 -0.06221164  0.21571543]
 [ 0.15122805 -0.025669   -0.02013773]]
[[[0.00522627 0.         0.        ]
  [0.         0.00440787 0.        ]
  [0.         0.         0.00354443]]

 [[0.04229854 0.         0.        ]
  [0.         0.01138309 0.        ]
  [0.         0.         0.01798454]]

 [[0.04326808 0.         0.        ]
  [0.         0.01239873 0.        ]
  [0.         0.         0.02195267]]

 [[0.00628779 0.         0.        ]
  [0.         0.00593941 0.        ]
  [0.         0.         0.00596429]]]
[1.0000000e+000 0.0000000e+000 4.7540738e-136 0.0000000e+000]
[[0.96591175 0.00939899 0.00977982 0.01490944]
 [0.01243545 0.97258078 0.00266294 0.01232084]
 [0.01512916 0.00241278 0.97179537 0.01066269]
 [0.02925115 0.01265147 0.01235118 0.9457462 ]]
(648000,)


In [ ]:
for k in range(train.shape[1]):
    fig, axs = plt.subplots(model_3.n_components+1, sharex=True, sharey=True)
    colours = cm.rainbow(np.linspace(0, 1, model_3.n_components))
    time = np.arange(0,100, 1/100)
    axs[0].plot(time, train[:10000,k])
    axs[0].set_title("EEG train data for channel %d " % k)
    axs[0].grid(True)
    train_k = train[:,k]
    
    for i  in range(1,model_3.n_components+1):
        mask = z == i-1
        #print(time[mask])
        axs[i].plot(time[mask], train_k[mask], ".-", c=colours[i-1])
        axs[i].set_title("#{0} hidden state".format(i-1))

        axs[i].grid(True)
    

    plt.show()

In [ ]:
from sklearn.utils import check_random_state

L=len(test) # We would like to predict the following 15 days' trend
Niter = 10 # A hyper parameter of generating samples

warnings.filterwarnings('ignore')
eeg0 = train
eeg2 = np.copy(train)

startprob_cdf = np.cumsum(model_3.startprob_)
transmat_cdf = np.cumsum(model_3.transmat_, axis=1)
random_state = model_3.random_state

rs = check_random_state(None)

for l in range(L):
    eeg2 = np.append(eeg2,[[0,0,0]],axis=0) # Add a pair of empty (d,v)
    true_eeg = np.copy(eeg0[:len(train)+l])
    #print(true_eeg.shape)
    state_seq = model_3.predict(true_eeg)
    previous_state = state_seq[-1]

    maxLL = -1e10
    for n in range(Niter):
        currstate = (transmat_cdf[previous_state]> rs.rand() ).argmax() # Go through transmat to get a new state

        new_sample = model_3._generate_sample_from_state(currstate) # generate from the new state
        tmp_eeg = np.copy(true_eeg)
        tmp_eeg = np.append(tmp_eeg,[new_sample],axis=0) # Append the new_sample for score
        tmp_maxLL = model_3.score(tmp_eeg) # 
        if tmp_maxLL > maxLL :

                maxLL = tmp_maxLL
                eeg2[-1] = new_sample[0]


In [ ]:
#print('Training duration (s) : ', time.time() - global_start_time)
true_data = np.zeros((3,len(eeg2)))
pred_data = eeg2.T
y_train = train.T
y_test = test.T
cut_off = int(0.9*(true_data.shape[1]))
for i in range(true_data.shape[0]): 
    plot_results(pred_data[i, :500], y_train[i,:500], False)
    true_data[i,:] = np.concatenate([y_train[i,:],y_test[i,:]])
    plot_results(pred_data[i,:], true_data[i,:], True)
    plot_results(pred_data[i,cut_off:], y_test[i, :], False)

In [ ]:
import pickle 
tot_time_length = 10000
#seg_time_length = 10000 - seq_len
nchannels = pred_data.shape[0]
params = {'tot_time_length': tot_time_length, 'cut_off': cut_off, 'nchannels': nchannels}
data = {'true_data': true_data, 'pred_data': pred_data}
with open('LSTM/output-data/3channel_HMM_data_SC4002E0-PSG.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(params, f)
    pickle.dump(scalers, f)
    pickle.dump(pred_data, f)
    pickle.dump(true_data, f)
f.close()